In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from scipy import stats

import os
import math

In [2]:
# Do this after connecting to Jupyter Notebook and setting up the server.  Use:
# jupyter notebook --ip=127.0.0.1 --port=8888

os.chdir('/home/poblivsig/Dropbox/horses2')
print(os.getcwd())

/home/poblivsig/Dropbox/horses2


In [3]:
## Grab the datasets and merge them.
df = pd.read_csv('data/horses_2020.csv')
dfr = pd.read_csv('data/races_2020.csv')
df_copy = df.copy()
dfn = df.merge(dfr, on='rid')


/home/poblivsig/Dropbox/horses2/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# TODO: Missing data
# Total: 149513
# saddle: 6
# Overweight: 144613
# outHandicap: 148159
# RPR: 72446
# TR: 101769
# OR: 69976

In [5]:
# TODO: Remove irrelevant data
# positionL (is it there) - Doesn't exist
# weightSt - We have kilos
# weightLb - We have kilos
# distance - Like 7f - Given in metres in 'metric'
# band - No idea what it is.
# prize - Given in different currencies
# currency - See above
# prizes - List of prizes - Combined (added together) in prize
# winningTime - Not relevant
# rclass - Contains word 'Class' at start.  Repeated as number in 'class'
# condition - Repeated as number in 'cond'

dfn = dfn.drop(['weightSt',
                'weightLb',
                'distance',
                'band',
                'prize',
                'currency',
                'prizes',
                'winningTime',
                'rclass',
                'condition',
                'time',
                'date'], axis=1)

In [6]:
## Get rid of invalid values on certain rows.

# For the following attributes, there are some that are nan.  Replace with ''
values = {'trainerName': '', 'jockeyName': '', 'headGear': '', 'mother': '', 'gfather': ''}
dfn.fillna(value=values, inplace=True)

In [7]:
## Let's do some encoding

label_encoder = LabelEncoder()
dfn['horseNameEnc'] = label_encoder.fit_transform(dfn['horseName'])
dfn['trainerNameEnc'] = label_encoder.fit_transform(dfn['trainerName'])
dfn['jockeyNameEnc'] = label_encoder.fit_transform(dfn['jockeyName'])
dfn['headGearEnc'] = label_encoder.fit_transform(dfn['headGear'])
dfn['fatherEnc'] = label_encoder.fit_transform(dfn['father'])
dfn['motherEnc'] = label_encoder.fit_transform(dfn['mother'])
dfn['gfatherEnc'] = label_encoder.fit_transform(dfn['gfather'])
dfn['countryCodeEnc'] = label_encoder.fit_transform(dfn['countryCode'])
dfn['courseEnc'] = label_encoder.fit_transform(dfn['course'])
dfn['titleEnc'] = label_encoder.fit_transform(dfn['title'])

# Need to keep correct order for age
label_mapping = {'2yo':0, '2yo+':1, '3yo':2, '3yo+':3, '2-3yo':4, '4yo':5, '4yo+':6, '3-4yo':7, '5yo':8, \
                 '5yo+':9, '4-5yo':10, '3-5yo':11, '6yo+':12, '3-6yo':13, '4-6yo':14,  '5-6yo':15, '4-7yo':16, \
                 '5-7yo':17, '4-8yo':18,'8yo+':20, '9yo+':21, '10yo+':22}

##%%
## Split hurdles column by fences and hurdles

## Create boolean series showing whether hurdles column contains the word
## 'hurdle'.
hurdles = dfn['hurdles'].str.contains('hurdle')

# Create new series containing 0 if no hurdle or grab the number if available.
# hurdle_count = pd.Series(dtype='int64')
hurdle_count = np.array([], dtype=int)

for count, hurdle in enumerate(hurdles):
    if hurdle and isinstance(dfn['hurdles'][count], str):
        ## Grab the number - i.e. first 'word' in string
       # and convert to int
        number = int(dfn['hurdles'][count].split(' ')[0])
    else:
        number = 0
    hurdle_count = np.append(hurdle_count, number)

    # hurdle_count = hurdle_count.set_value(count, number)
    if count % 1000 == 0:
        print(f'hurdles processed = {count}')

## Add Series to our dataset
dfn['hurdles_cnt'] = hurdle_count.tolist()

hurdles processed = 0
hurdles processed = 1000
hurdles processed = 2000
hurdles processed = 3000
hurdles processed = 4000
hurdles processed = 5000
hurdles processed = 6000
hurdles processed = 7000
hurdles processed = 8000
hurdles processed = 9000
hurdles processed = 10000
hurdles processed = 11000
hurdles processed = 12000
hurdles processed = 13000
hurdles processed = 14000
hurdles processed = 15000
hurdles processed = 16000
hurdles processed = 17000
hurdles processed = 18000
hurdles processed = 19000
hurdles processed = 20000
hurdles processed = 21000
hurdles processed = 22000
hurdles processed = 23000
hurdles processed = 24000
hurdles processed = 25000
hurdles processed = 26000
hurdles processed = 27000
hurdles processed = 28000
hurdles processed = 29000
hurdles processed = 30000
hurdles processed = 31000
hurdles processed = 32000
hurdles processed = 33000
hurdles processed = 34000
hurdles processed = 35000
hurdles processed = 36000
hurdles processed = 37000
hurdles processed = 38000

In [8]:
# Convert dist to floats
dfn['dist'] = pd.to_numeric(df['dist'], errors='coerce')

In [9]:
# Convert nans in relevant columns to 0
dfn['overWeight'] = dfn['overWeight'].fillna(0)
dfn['outHandicap'] = dfn['outHandicap'].fillna(0)

In [10]:
## Create boolean series showing whether hurdles column contains the word
## 'hurdle'.
# dfn['first'] = np.where(dfn.position == 1, True, False)

In [11]:
# # Create new series containing 0 if no hurdle or grab the number if available.
# ## Create boolean series showing whether hurdles column contains the word
# ## 'fence'.
# hurdle = dfn['hurdles'].str.contains('hurdles')
#
# # Create new series containing 0 if no hurdle or grab the number if available.
# # hurdle_count = pd.Series(dtype='int64')
# hurdle_count = np.array([], dtype=int)
# # hurdle_count = pd.Series(dtype='int64')
#
# for count, hurdle in enumerate(hurdles):
#     if hurdle and isinstance(dfn['hurdles'][count], str):
#         ## Grab the number - i.e. first 'word' in string
#        # and convert to int
#         number = int(dfn['hurdles'][count].split(' ')[0])
#     else:
#         number = 0
#     hurdle_count = np.append(hurdle_count, number)
#
#     # hurdle_count = hurdle_count.set_value(count, number)
#     if count % 1000 == 0:
#         print(f'hurdles processed = {count}')
#
# ## Add Series to our dataset
# dfn['hurdles_cnt'] = hurdle_count.tolist()

In [12]:
## Create boolean series showing whether fences column contains the word
## 'fence'.
fences = dfn['hurdles'].str.contains('fences')

# Create new series containing 0 if no hurdle or grab the number if available.
# hurdle_count = pd.Series(dtype='int64')
fence_count = np.array([], dtype=int)

for count, fence in enumerate(fences):
    if fence and isinstance(dfn['hurdles'][count], str):
        ## Grab the number - i.e. first 'word' in string
        # and convert to int
        number = int(dfn['hurdles'][count].split(' ')[0])
    else:
        number = 0
    fence_count = np.append(fence_count, number)

    # fence_count = fence_count.set_value(count, number)
    if count % 1000 == 0:
        print(f'fences processed = {count}')

## Add Series to our dataset
dfn['fences_cnt'] = fence_count.tolist()

fences processed = 0
fences processed = 1000
fences processed = 2000
fences processed = 3000
fences processed = 4000
fences processed = 5000
fences processed = 6000
fences processed = 7000
fences processed = 8000
fences processed = 9000
fences processed = 10000
fences processed = 11000
fences processed = 12000
fences processed = 13000
fences processed = 14000
fences processed = 15000
fences processed = 16000
fences processed = 17000
fences processed = 18000
fences processed = 19000
fences processed = 20000
fences processed = 21000
fences processed = 22000
fences processed = 23000
fences processed = 24000
fences processed = 25000
fences processed = 26000
fences processed = 27000
fences processed = 28000
fences processed = 29000
fences processed = 30000
fences processed = 31000
fences processed = 32000
fences processed = 33000
fences processed = 34000
fences processed = 35000
fences processed = 36000
fences processed = 37000
fences processed = 38000
fences processed = 39000
fences proces

In [13]:
# Get rid of all the columns that have been encoded
dfn = dfn.drop(['horseName'], axis=1)
dfn = dfn.drop(['trainerName'], axis=1)
dfn = dfn.drop(['jockeyName'], axis=1)
dfn = dfn.drop(['headGear'], axis=1)
dfn = dfn.drop(['father'], axis=1)
dfn = dfn.drop(['mother'], axis=1)
dfn = dfn.drop(['gfather'], axis=1)
dfn = dfn.drop(['countryCode'], axis=1)
dfn = dfn.drop(['course'], axis=1)
dfn = dfn.drop(['title'], axis=1)
dfn = dfn.drop(['hurdles'], axis=1)
dfn = dfn.drop(['ages'], axis=1)
dfn = dfn.drop(['positionL'], axis=1)
dfn = dfn.drop(['price'], axis=1)
dfn = dfn.drop(['position'],axis=1)

In [14]:
# Set missing data on relevant columns to their mean
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer = imputer.fit(dfn) #X[:, 1:])
# dfn = imputer.transform(dfn) #X[:, 1:])
imp = SimpleImputer(missing_values=math.nan, strategy="mean")
dfn["OR"] = imp.fit_transform(dfn[["OR"]]).ravel()
dfn["RPR"] = imp.fit_transform(dfn[["RPR"]]).ravel()
dfn["TR"] = imp.fit_transform(dfn[["TR"]]).ravel()
dfn["dist"] = imp.fit_transform(dfn[["dist"]]).ravel()
dfn["saddle"] = imp.fit_transform(dfn[["saddle"]]).ravel()

In [15]:
## Save result to disk
dfn.to_csv('data/proc_races.csv')

In [ ]:
# TODO: Remove all handicap rows. Will try with first and then without
# These are horses that run in handicap
# races and have weight added.  The skill here is in guessing if the
# handicap is too much or too little.

In [ ]:
######################### HELPER CODE #################################

In [ ]:
## Get info about horses
print(f'Shape\n\n{dfn.shape}')
print(f'Columns\n\n{dfn.columns}')
print(f'dtypes\n\n{dfn.dtypes}')
pd.set_option('display.max_columns', None)
print(f'Description\n\n{dfn.describe()}')
print(dfn.sample(n=1))
pd.set_option('display.max_columns', 5)

In [82]:
## Show unique values for columns missing data
# print("headGear Values")
# print(dfn.headGear.unique())
# print(f'No of values = {len(dfn.headGear.unique())}')

# print("horseName Values")
# print(f'No of values = {len(dfn.horseName.unique())}')

# print("trainerName Values")
# print(f'No of values = {len(dfn.trainerName.unique())}')

# print("jockeyName Values")
# print(f'No of values = {len(dfn.jockeyName.unique())}')

print("mother Values")
print(f'No of values = {len(dfn.mother.unique())}')

print("father Values")
print(f'No of values = {len(dfn.father.unique())}')

print("ages Values")
print(dfn.ages.unique())
print(f'No of values = {len(dfn.ages.unique())}')

print("hurdles Values")
print(dfn.hurdles.unique())
print(f'No of values = {len(dfn.hurdles.unique())}')


mother Values


AttributeError: 'DataFrame' object has no attribute 'mother'

In [ ]:
# Check the trainerName for numbers.  This checks column for floats and outputs the values
for count, trainer in enumerate(dfn['trainerName']):
    if isinstance(trainer, float):
        print(trainer)

In [65]:
# Check whether positions contains all numbers
for count, pos in enumerate(dfn['position']):
    if math.isnan(pos):
    # if not isinstance(pos, int):
        print(pos)

In [50]:
for (columnName, _) in dfn.iteritems():
    for count, pos in enumerate(dfn[columnName]):
        if pos > 10000000:
        # if math.isinf(pos):
        # if not isinstance(pos, int) and not isinstance(pos, float):
            print(columnName, pos)
